# Gemma 3 D-Fire Dataset Fine-tuning

Fine-tune Gemma 3 on the D-Fire dataset for fire and smoke detection. This notebook demonstrates how to:

1. Load and preprocess D-Fire dataset images with labels
2. Create vision-text pairs for fire/smoke detection
3. Fine-tune the multimodal model on the dataset
4. Configure the number of images used for training

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!

<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### D-Fire Dataset Fine-tuning Configuration

**Key Features:**
- 🔥 **Fire/Smoke Detection**: Fine-tune on D-Fire dataset for fire and smoke detection
- 🎯 **Configurable Dataset Size**: Set `MAX_IMAGES_PER_CATEGORY` to control training data size
- 🖼️ **Vision Fine-tuning**: Enabled vision layers for multimodal learning
- 📊 **Balanced Categories**: Automatically balances fire, smoke, both, and none categories
- 🔄 **Adaptive Training**: Training steps adjust automatically based on dataset size

**Configuration Parameters:**
- `DATASET_PATH`: Path to your D-Fire dataset
- `MAX_IMAGES_PER_CATEGORY`: Number of images per category (default: 200)
- `DETECTION_PROMPT`: The prompt used for fire/smoke detection

### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !uv pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
!uv pip show transformers triton torch xformers

In [ ]:
# Configuration for D-Fire dataset
DATASET_PATH = "../../data/D-Fire/"  # Update this path to your D-Fire dataset location
MAX_IMAGES_PER_CATEGORY = 3000  # Number of images to use per category (fire, smoke, both, none)

# Define the detection prompt and category mappings
DETECTION_PROMPT = "Is there fire or smoke visible in this image? Answer with: 'fire', 'smoke', 'both', or 'none'."
CATEGORY_LABELS = {
    "fire": "fire",
    "smoke": "smoke", 
    "both": "both",
    "none": "none"
}

# Verify the D-Fire dataset structure
import os
print("Checking D-Fire dataset structure...")
if os.path.exists(DATASET_PATH):
    print(f"✅ Found D-Fire folder at: {DATASET_PATH}")
    
    # Check for train and test folders
    train_path = os.path.join(DATASET_PATH, "train")
    test_path = os.path.join(DATASET_PATH, "test")
    
    if os.path.exists(train_path):
        print(f"✅ Found train folder: {train_path}")
        train_images = os.path.join(train_path, "images")
        train_labels = os.path.join(train_path, "labels")
        if os.path.exists(train_images) and os.path.exists(train_labels):
            print(f"✅ Found train/images and train/labels folders")
            print(f"   - Images: {len(os.listdir(train_images))} files")
            print(f"   - Labels: {len(os.listdir(train_labels))} files")
        else:
            print("⚠️ Missing train/images or train/labels folders")
    else:
        print("❌ Train folder not found")
    
    if os.path.exists(test_path):
        print(f"✅ Found test folder: {test_path}")
        test_images = os.path.join(test_path, "images")
        test_labels = os.path.join(test_path, "labels")
        if os.path.exists(test_images) and os.path.exists(test_labels):
            print(f"✅ Found test/images and test/labels folders")
            print(f"   - Images: {len(os.listdir(test_images))} files")
            print(f"   - Labels: {len(os.listdir(test_labels))} files")
        else:
            print("⚠️ Missing test/images or test/labels folders")
    else:
        print("❌ Test folder not found")

else:
    print(f"❌ D-Fire dataset not found at: {DATASET_PATH}")
    print("Please ensure your D-Fire dataset is connected to kaggle")
    print("Expected structure:")
    print("/kaggle/input/d-fire/")
    print("├── train/")
    print("│   ├── images/")
    print("│   └── labels/")
    print("└── test/")
    print("    ├── images/")
    print("    └── labels/")

print(f"\nConfigured to use {MAX_IMAGES_PER_CATEGORY} images per category")
print(f"Detection prompt: {DETECTION_PROMPT}")


### Unsloth

In [ ]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Llama-3.2-11B-Vision-Instruct-bnb-4bit", # Llama 3.2 vision support
    "unsloth/Llama-3.2-11B-Vision-bnb-4bit",
    "unsloth/Llama-3.2-90B-Vision-Instruct-bnb-4bit", # Can fit in a 80GB card!
    "unsloth/Llama-3.2-90B-Vision-bnb-4bit",

    "unsloth/Pixtral-12B-2409-bnb-4bit",              # Pixtral fits in 16GB!
    "unsloth/Pixtral-12B-Base-2409-bnb-4bit",         # Pixtral base model

    "unsloth/Qwen2-VL-2B-Instruct-bnb-4bit",          # Qwen2 VL support
    "unsloth/Qwen2-VL-7B-Instruct-bnb-4bit",
    "unsloth/Qwen2-VL-72B-Instruct-bnb-4bit",

    "unsloth/llava-v1.6-mistral-7b-hf-bnb-4bit",      # Any Llava variant works!
    "unsloth/llava-1.5-7b-hf-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, processor = FastVisionModel.from_pretrained(
    "unsloth/gemma-3-4b-pt",
    load_in_4bit = False, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

We now add LoRA adapters for parameter efficient fine-tuning, allowing us to train only 1% of all model parameters efficiently.

**[NEW]** We also support fine-tuning only the vision component, only the language component, or both. Additionally, you can choose to fine-tune the attention modules, the MLP layers, or both!

In [ ]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True, # False if not finetuning vision layers
    finetune_language_layers   = True, # False if not finetuning language layers
    finetune_attention_modules = True, # False if not finetuning attention layers
    finetune_mlp_modules       = True, # False if not finetuning MLP layers

    r = 16,                           # The larger, the higher the accuracy, but might overfit
    lora_alpha = 16,                  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,               # We support rank stabilized LoRA
    loftq_config = None,               # And LoftQ
    target_modules = "all-linear",    # Optional now! Can specify a list if needed
    modules_to_save=[
        "lm_head",
        "embed_tokens",
    ],
)

<a name="Data"></a>
### Data Prep
We'll use the D-Fire dataset for fire and smoke detection. The objective is to create a vision-text model that can accurately identify fire, smoke, both, or none in images. This is particularly useful for fire detection and emergency response applications.

The D-Fire dataset contains images categorized into:
- **Fire**: Images containing fire
- **Smoke**: Images containing smoke
- **Both**: Images containing both fire and smoke
- **None**: Images containing neither fire nor smoke

In [ ]:
# D-Fire Dataset Loader
import os
import random
from pathlib import Path
from datasets import Dataset
from PIL import Image
import pandas as pd
from matplotlib import pyplot as plt

class DFireDatasetLoader:
    """Custom D-Fire dataset loader for fire and smoke detection."""
    
    def __init__(self, dataset_path: str):
        self.dataset_path = Path(dataset_path)
        self.train_path = self.dataset_path / "train"
        self.test_path = self.dataset_path / "test"
    
    def _load_yolo_labels(self, labels_dir: Path) -> dict:
        """Load YOLO format labels from directory."""
        labels = {}
        for label_file in labels_dir.glob("*.txt"):
            image_name = label_file.stem
            annotations = []
            
            if label_file.stat().st_size > 0:
                with open(label_file, 'r') as f:
                    for line in f:
                        parts = line.strip().split()
                        if len(parts) >= 5:
                            class_id = int(parts[0])
                            annotations.append({
                                'class_id': class_id,
                                'class_name': 'smoke' if class_id == 0 else 'fire'
                            })
            labels[image_name] = annotations
        return labels
    
    def _categorize_images(self, labels: dict) -> dict:
        """Categorize images based on their annotations."""
        categories = {'fire': [], 'smoke': [], 'both': [], 'none': []}
        
        for image_name, annotations in labels.items():
            if not annotations:
                categories['none'].append(image_name)
            else:
                has_fire = any(ann['class_name'] == 'fire' for ann in annotations)
                has_smoke = any(ann['class_name'] == 'smoke' for ann in annotations)
                
                if has_fire and has_smoke:
                    categories['both'].append(image_name)
                elif has_fire:
                    categories['fire'].append(image_name)
                elif has_smoke:
                    categories['smoke'].append(image_name)
                else:
                    categories['none'].append(image_name)
        
        return categories
    
    def load_dataset(self, split: str = "train", max_per_category: int = 200, seed: int = 42) -> list:
        """Load dataset with specified number of images per category."""
        random.seed(seed)
        
        split_path = self.train_path if split == "train" else self.test_path
        images_dir = split_path / "images"
        labels_dir = split_path / "labels"
        
        # Load labels and categorize
        labels = self._load_yolo_labels(labels_dir)
        categories = self._categorize_images(labels)
        
        # Get available images
        available_images = set(img.stem for img in images_dir.glob("*.[jp][pn]g"))
        
        # Filter and sample
        sample_data = []
        for category, image_names in categories.items():
            available_names = [name for name in image_names if name in available_images]
            if available_names:
                sampled = random.sample(available_names, min(max_per_category, len(available_names)))
                for image_name in sampled:
                    image_path = images_dir / f"{image_name}.jpg"
                    if not image_path.exists():
                        image_path = images_dir / f"{image_name}.png"
                    
                    if image_path.exists():
                        sample_data.append({
                            'image_path': str(image_path),
                            'category': category,
                            'label': CATEGORY_LABELS[category],
                            'image_name': image_name
                        })
        
        return sample_data

# Load the D-Fire dataset
if os.path.exists(DATASET_PATH):
    loader = DFireDatasetLoader(DATASET_PATH)
    dataset_samples = loader.load_dataset(split="train", max_per_category=MAX_IMAGES_PER_CATEGORY)
    print(f"Loaded {len(dataset_samples)} samples from D-Fire dataset")
    
    # Show distribution
    from collections import Counter
    category_counts = Counter([sample['category'] for sample in dataset_samples])
    print(f"Category distribution: {dict(category_counts)}")
else:
    print(f"Dataset path not found: {DATASET_PATH}")
    print("Please update DATASET_PATH to point to your D-Fire dataset location")
    dataset_samples = []

Let's take an overview of the D-Fire dataset. We'll examine a sample image and its corresponding label.

In [ ]:
if dataset_samples:
    print(f"Dataset Statistics:")
    print(f"Total samples: {len(dataset_samples)}")
    print(f"Categories: {list(CATEGORY_LABELS.keys())}")
    print(f"Max images per category: {MAX_IMAGES_PER_CATEGORY}")
    print(f"\nSample data structure:")
    sample = dataset_samples[0]
    print(f"Image path: {sample['image_path']}")
    print(f"Category: {sample['category']}")
    print(f"Label: {sample['label']}")
    print(f"Image name: {sample['image_name']}")
else:
    print("No dataset samples available. Please check your dataset path.")

In [ ]:
if dataset_samples:
    # Display a sample image
    sample = dataset_samples[2] if len(dataset_samples) > 2 else dataset_samples[0]
    img = Image.open(sample['image_path']).convert("RGB")
    print(f"Sample Image - Category: {sample['category']}, Label: {sample['label']}")
    plt.imshow(img)
else:
    print("No sample image available.")

In [ ]:
if dataset_samples:
    sample = dataset_samples[2] if len(dataset_samples) > 2 else dataset_samples[0]
    print(f"Detection Label: {sample['label']}")
    print(f"Detection Prompt: {DETECTION_PROMPT}")
else:
    print("No sample label available.")

Let's see another example from our D-Fire dataset!

In [ ]:
if dataset_samples and len(dataset_samples) > 3:
    # Show a different category example
    sample = dataset_samples[3]
    img = Image.open(sample['image_path']).convert("RGB")
    print(f"Another Sample - Category: {sample['category']}, Label: {sample['label']}")
    plt.imshow(img)
else:
    print("Additional sample not available.")

To format the dataset, all vision fine-tuning tasks should follow this format:

```python
[
    {
        "role": "user",
        "content": [
            {"type": "text", "text": DETECTION_PROMPT},
            {"type": "image", "image": sample["image"]},
        ],
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": sample["label"]},
        ],
    },
]
```

In [ ]:
def convert_to_conversation(sample):
    """Convert D-Fire sample to conversation format."""
    # Load the image
    img = Image.open(sample['image_path']).convert("RGB")
    
    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": DETECTION_PROMPT},
                {"type": "image", "image": img},
            ],
        },
        {"role": "assistant", "content": [{"type": "text", "text": sample["label"]}]},
    ]
    return {"messages": conversation}
pass

Let's convert the dataset into the "correct" format for finetuning:

In [ ]:
if dataset_samples:
    converted_dataset = [convert_to_conversation(sample) for sample in dataset_samples]
    print(f"Converted {len(converted_dataset)} samples to conversation format")
else:
    print("No dataset samples to convert.")
    converted_dataset = []

The first example is now structured like below:

In [ ]:
if converted_dataset:
    print("First converted conversation:")
    print(converted_dataset[0])
else:
    print("No converted conversations available.")

Lets take the Gemma 3 instruction chat template and use it in our base model

In [ ]:
from unsloth import get_chat_template

processor = get_chat_template(
    processor,
    "gemma-3"
)

Before fine-tuning, let us evaluate the base model's performance. We do not expect strong results, as it has not encountered this chat template before.

In [ ]:
FastVisionModel.for_inference(model)  # Enable for inference!

if dataset_samples:
    # Test with a D-Fire sample
    sample = dataset_samples[2] if len(dataset_samples) > 2 else dataset_samples[0]
    image = Image.open(sample['image_path']).convert("RGB")
    
    messages = [
        {
            "role": "user",
            "content": [{"type": "image"}, {"type": "text", "text": DETECTION_PROMPT}],
        }
    ]
    input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(
        image,
        input_text,
        add_special_tokens=False,
        return_tensors="pt",
    ).to("cuda")

    from transformers import TextStreamer

    text_streamer = TextStreamer(processor, skip_prompt=True)
    print(f"Testing with image from category: {sample['category']}")
    print(f"Expected answer: {sample['label']}")
    print("Model response:")
    result = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                            use_cache=True, temperature = 1.0, top_p = 0.95, top_k = 64)
else:
    print("No dataset samples available for inference testing.")

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

We use our new `UnslothVisionDataCollator` which will help in our vision finetuning setup.

In [ ]:
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

FastVisionModel.for_training(model) # Enable for training!

trainer = SFTTrainer(
    model=model,
    train_dataset=converted_dataset,
    processing_class=processor.tokenizer,
    data_collator=UnslothVisionDataCollator(model, processor),
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        gradient_checkpointing = True,

        # use reentrant checkpointing
        gradient_checkpointing_kwargs = {"use_reentrant": False},
        max_grad_norm = 0.3,              # max gradient norm based on QLoRA paper
        warmup_ratio = 0.03,
        max_steps = 100,
        # num_train_epochs = 2,          # Set this instead of max_steps for full training runs
        learning_rate = 2e-4,
        logging_steps = 1,
        save_strategy="steps",
        optim = "adamw_torch_fused",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",             # For Weights and Biases

        # You MUST put the below items for vision finetuning:
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        dataset_num_proc = 2,
        max_seq_length = 2048,
    )
)

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model! You can modify the instruction and input—just leave the output blank.

We'll use the best hyperparameters for inference on Gemma: `top_p=0.95`, `top_k=64`, and `temperature=1.0`.

In [ ]:
FastVisionModel.for_inference(model)  # Enable for inference!

if dataset_samples and len(dataset_samples) > 10:
    # Test with a D-Fire sample
    sample = dataset_samples[10]
    image = Image.open(sample['image_path']).convert("RGB")
    
    messages = [
        {
            "role": "user",
            "content": [{"type": "image"}, {"type": "text", "text": DETECTION_PROMPT}],
        }
    ]

    input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(
        image,
        input_text,
        add_special_tokens=False,
        return_tensors="pt",
    ).to("cuda")

    from transformers import TextStreamer

    text_streamer = TextStreamer(processor, skip_prompt=True)
    print(f"Testing with image from category: {sample['category']}")
    print(f"Expected answer: {sample['label']}")
    print("Model response:")
    result = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                            use_cache=True, temperature = 1.0, top_p = 0.95, top_k = 64)
else:
    print("Not enough dataset samples available for inference testing.")

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, use Hugging Face’s `push_to_hub` for online saving, or `save_pretrained` for local storage.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
processor.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# processor.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastVisionModel

    model, processor = FastVisionModel.from_pretrained(
        model_name="lora_model",  # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit=True,  # Set to False for 16bit LoRA
    )
    FastVisionModel.for_inference(model)  # Enable for inference!

FastVisionModel.for_inference(model)  # Enable for inference!

if dataset_samples and len(dataset_samples) > 1:
    sample = dataset_samples[1]
    image = Image.open(sample['image_path']).convert("RGB")
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": DETECTION_PROMPT,
                },
                {
                    "type": "image",
                },
            ],
        },
    ]
    input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(
        image,
        input_text,
        add_special_tokens=False,
        return_tensors="pt",
    ).to("cuda")

    from transformers import TextStreamer

    text_streamer = TextStreamer(processor.tokenizer, skip_prompt=True)
    print(f"Testing with image from category: {sample['category']}")
    print(f"Expected answer: {sample['label']}")
    print("Model response:")
    _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                       use_cache=True, temperature = 1.0, top_p = 0.95, top_k = 64)
else:
    print("Dataset samples not available for inference testing.")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Select ONLY 1 to save! (Both not needed!)

# Save locally to 16bit
if True: model.save_pretrained_merged("unsloth_finetune", processor,)

# To export and save to your Hugging Face account
if False: model.push_to_hub_merged("YOUR_USERNAME/unsloth_finetune", processor, token = "PUT_HERE")

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
